# Create segments for general use.

Define workspace. 

In [1]:
# Define workspace
import arcpy
import os

# fgdb in the `data` folder.
db = arcpy.env.workspace = r"C:\Users\AhjungKim\Documents\ArcGIS\Projects\sun_cloud_pro\general_segments.gdb"
arcpy.env.overwriteOutput = True


distance = 5


Reusable functions.

In [2]:
from arcgis.gis import GIS
gis = GIS('pro')
from arcgis.features import FeatureLayer

# save feature layer to db
def save_fl(db, url, outname):
    fl = FeatureLayer(url)
    featureset = fl.query()
    featureset.save(db, outname)

# project in_data to match the target_data

def project(in_data, target_data, out_name):
    #TODO: do not hardcode this
    target_data = 'PM3_For_HPMS_2_ExportFeature'

    in_sr = arcpy.Describe(in_data).spatialReference
    target_sr = arcpy.Describe(target_data).spatialReference
   
    if len(arcpy.ListTransformations (in_sr, target_sr))>0:
        datum_conversion = arcpy.ListTransformations (in_sr, target_sr)[0]
    else: 
        datum_conversion = ''



    arcpy.Project_management(
        in_dataset = in_data, 
        out_dataset = out_name,
        out_coor_system = target_sr,
        transform_method = datum_conversion)

Save the feature layers locally

In [3]:
routes_url = 'https://services6.arcgis.com/clPWQMwZfdWn4MQZ/arcgis/rest/services/Sun_Cloud_Routes/FeatureServer/92' 

save_fl(db, routes_url, 'sun_cloud_routes')


Project the data.

In [4]:
# project in_data to match the target_data

def project(in_data, target_sr, out_name):

    in_sr = arcpy.Describe(in_data).spatialReference
    print(in_sr.name)
    tr = arcpy.ListTransformations (in_sr, target_sr)
    print(tr)
    if(len(tr)>0):
        datum_conversion = arcpy.ListTransformations (in_sr, target_sr)[0]

        arcpy.Project_management(
        in_dataset = in_data, 
        out_dataset = out_name,
        out_coor_system = target_sr,
        transform_method = datum_conversion)
    else:
        arcpy.FeatureClassToFeatureClass_conversion(in_data, db, out_name)

In [5]:
in_fc = 'sun_cloud_routes'
target_sr = arcpy.SpatialReference(2223)
out_fc = 'sun_cloud_routes_project'

project(in_fc, target_sr, out_fc)

NAD_1983_StatePlane_Arizona_Central_FIPS_0202_Feet_Intl
[]


### Split at major intersection
major == functional class is 1-7, 11-17 per Yousef Dana

From the original routes layer (NOT the segmented route), delete non-major routes.

Create major intersection points. 

In [6]:
# create a backup of sun cloud routes
arcpy.conversion.ExportFeatures("sun_cloud_routes_project", "sun_cloud_routes_project_backup")
route_raw = 'sun_cloud_routes_project'
sql = """functional_class BETWEEN 1 AND 7 OR functional_class BETWEEN 11 AND 17"""
selected = arcpy.SelectLayerByAttribute_management(route_raw, "NEW_SELECTION", sql, "INVERT")
count = int(arcpy.GetCount_management(selected)[0])
print (count)

if count > 0:
    arcpy.DeleteFeatures_management(selected)

0


Find the real intersections. 
Reference: unsplit the original route to find the real intersections (https://support.esri.com/en/technical-article/000025044)

In [7]:
route_raw = 'sun_cloud_routes_project'
arcpy.management.UnsplitLine(route_raw, "unsplit_route")

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\general_segments.gdb\\unsplit_route'>

In [8]:
# Run the Intersect tool to create points at interaction.  Note that this layer is used to create the Sunc Cloud junciton layer.
arcpy.analysis.Intersect("sun_cloud_routes_project", "major_intersection", "ALL", None, "POINT")

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\general_segments.gdb\\major_intersection'>

Covert multipart point to single part.

In [9]:
arcpy.management.MultipartToSinglepart("major_intersection", "major_intersec_single")

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\general_segments.gdb\\major_intersec_single'>

Split at major intersection.

In [10]:
# split the segmented layer at the major intersection point
inFeatures = 'sun_cloud_routes_project'
pointFeatures = 'major_intersec_single'
splitresult = 'split_intersection'
arcpy.SplitLineAtPoint_management(inFeatures, pointFeatures, splitresult, '1 Meters')

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\general_segments.gdb\\split_intersection'>

### If interstate or freeway is longer than 1 miles, split at one-mile point.

Add length field (miles).

In [11]:
# add length field in miles
in_f = "split_intersection"
geom_prop = "Length_Mi LENGTH_GEODESIC"
unit = "MILES_US"
result = arcpy.management.CalculateGeometryAttributes(in_f, geom_prop, unit)
print(result)

C:\Users\AhjungKim\Documents\ArcGIS\Projects\sun_cloud_pro\general_segments.gdb\split_intersection


Create a 10 meter buffer of urban area. 

In [12]:
outname = 'urban_area_Buffer'
if arcpy.Exists(outname)== False:
    arcpy.analysis.Buffer("urban_area", outname, "35 Feet", "FULL", "ROUND", "NONE", None, "PLANAR")
else:
    print ('buffer already exists...')
    

buffer already exists...


Select routes that are longer than defined distnace, but not within the buffer. 

In [13]:
in_f = 'split_intersection'

print("Length_Mi >{}".format(distance))
selected = arcpy.SelectLayerByAttribute_management(in_f, "NEW_SELECTION", "Length_Mi >{}".format(distance))
subselect = arcpy.management.SelectLayerByLocation(selected, "WITHIN", "urban_area_Buffer", None, "REMOVE_FROM_SELECTION", "NOT_INVERT")

arcpy.MakeFeatureLayer_management(subselect, 'select_layer')
count = int(arcpy.GetCount_management('select_layer')[0])
print(count)

if (count>0):
    split_points = 'mile_points_{}'.format(distance)
    dist = '{} miles'.format(distance)
    arcpy.GeneratePointsAlongLines_management(
        Input_Features='select_layer', 
        Output_Feature_Class=split_points, 
        Point_Placement='DISTANCE',
        Distance=dist)
    

Length_Mi >5
209


Split at one-mile point.

In [14]:
# split the segmented layer at the major intersection point
inFeatures = 'split_intersection'
pointFeatures = split_points
splitresult = "sun_cloud_segmented_route"
searchRadius = "1 Feet"


arcpy.SplitLineAtPoint_management(inFeatures, pointFeatures, splitresult, searchRadius)

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\general_segments.gdb\\sun_cloud_segmented_route'>

Delete short (<250') segments.

In [15]:
# add length field in feet

in_f = "sun_cloud_routes_project"
geom_prop = "Length_Ft LENGTH"
unit = "FEET_INT"
result = arcpy.management.CalculateGeometryAttributes(in_f, geom_prop, unit)

where = 'Length_Ft < 250'
selected = arcpy.SelectLayerByAttribute_management('sun_cloud_routes_project', 'NEW_SELECTION', where)
count = int(arcpy.GetCount_management(selected)[0])
print(count)
if(count>0):
    arcpy.DeleteFeatures_management(selected)

17


### Clean up fields.

Delete fields if not in the original route layer.

In [16]:
# get required fields
fields = arcpy.ListFields('sun_cloud_routes')
route_fields = [field.name for field in fields if field.required is False]
route_fields

['route_id', 'functional_class']

In [17]:
data = 'sun_cloud_segmented_route'
fields = arcpy.ListFields(data)
f_names = [field.name for field in fields if field.required is False]

# delete one by on due to arcpy bug
for f in f_names:
    if f not in route_fields:
        arcpy.DeleteField_management(data, f)

Decode functional class. 

In [19]:
fc='sun_cloud_segmented_route'
arcpy.AddField_management(fc, 'functional_classification_code', 'SHORT')

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\general_segments.gdb\\sun_cloud_segmented_route'>

In [21]:
arcpy.management.CalculateField(
    in_table=fc,
    field="functional_classification_code",
    expression="!functional_class!",
    expression_type="PYTHON3",

    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\general_segments.gdb\\sun_cloud_segmented_route'>

In [23]:
arcpy.AddField_management(fc, 'functional_classification', 'TEXT')
arcpy.management.CalculateField(
    in_table=fc,
    field="functional_classification",
    expression="!functional_class!",
    expression_type="PYTHON3",

    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\general_segments.gdb\\sun_cloud_segmented_route'>

In [24]:
arcpy.management.CreateDomain(db, "functional class", 
                              "functional class", "TEXT", "CODED")

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\general_segments.gdb'>

In [27]:
fc_dict = {
    1 : 'Interstate',
    2 : 'Other Freeways and Expressways',
    3 : 'Other Principal Arterial',
    4 : 'Minor Arterial',
    5 : 'Major Collector',
    6 : 'Minor Collector',
    7:  'Local'
}

In [30]:
for i in fc_dict:
    arcpy.AddCodedValueToDomain_management(db, 'functional class', i, fc_dict[i])


In [32]:
arcpy.management.AssignDomainToField(fc, 'functional_classification', 'functional class')

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\general_segments.gdb\\sun_cloud_segmented_route'>

In [33]:
target_fields = [
    ('functional_classification', 'functional_classification', 'Functional Classification'),
    ('functional_classification_code', 'functional_classification_code', 'Functional Classification_code'),

]
for current, new, alias in target_fields:
    arcpy.management.AlterField(fc, current, new, alias)

arcpy.DeleteField_management(fc, 'functional_class')

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\general_segments.gdb\\sun_cloud_segmented_route'>